# Anomly detection for IoT 

Preprocessed dataset for network based intrusion detection system in Iot Devices. 
Ultrasonic Sensor with Arduino and NodeMCU used to monitor the network and collect the network logs. 
NodeMCU with ESP8266 wifi module was used to send data to the server via wifi.

The normality column can have the following values:
- 0 - Normal
- 1 - Wrong Setup
- 2 - DDOS
- 3 - Data type probing
- 4 - Scan Attack
- 5 - Man in the middle

For more detail, check [here](https://www.kaggle.com/datasets/speedwall10/iot-device-network-logs).


In [1]:
%matplotlib inline
import numpy as np
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 5]
plt.rcParams["figure.autolayout"] = True
plt.rcParams.update({'font.size': 22})

In [2]:
df = pl.read_csv('dataset/IoT-DNL.csv.gz')
#df = df.sample(int(df.shape[0]*0.2))
df.head()

frame.number,frame.time,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.proto,ip.len,tcp.len,tcp.srcport,tcp.dstport,Value,normality
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,i64
1,123722736684743,54,87971959760497,167275820076079,192168035,1921680121,6.0,40.0,0.0,49279.0,80.0,-99.0,0
2,123722736773147,62,87971959760497,167275820076079,192168035,1921680121,6.0,48.0,0.0,56521.0,80.0,-99.0,0
3,123722736824792,62,167275820076079,87971959760497,1921680121,192168035,6.0,48.0,0.0,80.0,56521.0,-99.0,0
4,123722736836228,54,167275820076079,87971959760497,1921680121,192168035,6.0,40.0,0.0,80.0,49279.0,-99.0,0
5,123722749684991,54,87971959760497,167275820076079,192168035,1921680121,6.0,40.0,0.0,56521.0,80.0,-99.0,0


In [3]:
df.describe()

statistic,frame.number,frame.time,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.proto,ip.len,tcp.len,tcp.srcport,tcp.dstport,Value,normality
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0,477426.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",52917.357471,1.2566e14,120.658661,1.2941e14,1.6078e14,8.6227e8,1.2069e9,2.858263,97.2428,60.844678,23722.759349,4528.371894,0.564996,2.489808
"""std""",32439.729155,2.0642e12,88.273425,4.4788e13,5.0725e13,2.0198e9,3.6758e9,3.284435,97.168551,87.68277,27906.683645,15426.452846,3188.911857,1.706533
"""min""",1.0,1.2372e14,42.0,3.7560e13,1.1011e12,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-99.0,0.0
"""25%""",27547.0,1.2434e14,42.0,8.7972e13,1.3991e14,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-5.0,1.0
"""50%""",47329.0,1.2491e14,98.0,1.1043e14,1.6728e14,1.92168035e8,1.9217e9,6.0,84.0,0.0,0.0,0.0,-3.0,2.0
"""75%""",78486.0,1.2565e14,176.0,1.6728e14,1.6728e14,1.9217e9,1.9217e9,6.0,162.0,110.0,55068.0,80.0,-2.0,4.0
"""max""",125158.0,1.3051e14,3484.0,2.0707e14,2.8147e14,1.7222e11,2.5526e11,17.0,3470.0,3418.0,65534.0,65534.0,2.202219e6,5.0
